# DataFrameIt - Exemplo 03: Tratamento de Erros e Retry

Este notebook demonstra como o DataFrameIt lida com erros e como você pode analisar e recuperar de falhas no processamento.

**Conceitos demonstrados:**
- Sistema automático de retry com backoff exponencial
- Classificação inteligente de erros (recuperáveis vs não-recuperáveis)
- Coluna `_dataframeit_status` (processed, error, None)
- Coluna `_error_details` com detalhes de erros
- Configuração de retry customizado
- Estratégias de recuperação

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/02_error_handling.ipynb)

## 1. Instalação

In [ ]:
!pip install -q dataframeit[google]

## 2. Configuração da API Key

In [ ]:
import os

# Opção 1: Usando Google Colab Secrets (recomendado)
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# Opção 2: Inserir manualmente (descomente e preencha)
# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import pandas as pd
from dataframeit import dataframeit

## 4. Definir Modelo Pydantic

In [ ]:
class ProductCategory(BaseModel):
    """Estrutura para categorização de produtos."""

    categoria: Literal['eletronicos', 'roupas', 'alimentos', 'livros', 'outros'] = Field(
        ...,
        description="Categoria principal do produto"
    )

    subcategoria: str = Field(
        ...,
        description="Subcategoria mais específica"
    )

    preco_estimado: Literal['barato', 'medio', 'caro', 'premium'] = Field(
        ...,
        description="Faixa de preço estimada baseada na descrição"
    )

## 5. Definir Template

In [ ]:
TEMPLATE = """
Categorize o produto descrito abaixo.

Descrição do produto:
{texto}

Seja preciso na categorização.
"""

## 6. Criar Dados com Casos Problemáticos

Incluimos alguns casos que podem gerar erros:
- Textos muito curtos
- Textos vazios
- Textos ambíguos

In [ ]:
dados = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8],
    'texto': [
        "Smartphone Samsung Galaxy S23 Ultra 256GB com câmera de 200MP",     # OK
        "",                                                                   # VAZIO - pode causar erro
        "Livro 'Clean Code' de Robert C. Martin, programação",              # OK
        "x",                                                                  # MUITO CURTO - pode causar erro
        "Tênis Nike Air Max running profissional",                           # OK
        "Pizza margherita congelada 400g",                                   # OK
        "    ",                                                               # SÓ ESPAÇOS - pode causar erro
        "Fone de ouvido Bluetooth com cancelamento de ruído ativo"          # OK
    ]
}

df = pd.DataFrame(dados)

print("Dados originais (com casos problemáticos):")
df

## 7. Processar com Configuração de Retry

In [ ]:
df_resultado = dataframeit(
    df,
    ProductCategory,
    TEMPLATE,
    text_column='texto',
    max_retries=3,          # Tentar até 3 vezes (padrão: 3)
    base_delay=1.0,         # Começar com 1 segundo (padrão: 1.0)
    max_delay=10.0,         # Máximo 10 segundos entre tentativas (padrão: 30.0)
    resume=True
)

print("Processamento concluído!")

## 8. Analisar Status do Processamento

In [ ]:
# Verificar coluna de status automática
print("Status de cada linha:")
print(df_resultado[['id', '_dataframeit_status']].to_string(index=False))

# Contar status
print("\n" + "-" * 50)
print("Resumo:")
status_counts = df_resultado['_dataframeit_status'].value_counts()
total = len(df_resultado)

print(f"  Processadas com sucesso: {status_counts.get('processed', 0)}/{total}")
print(f"  Com erro: {status_counts.get('error', 0)}/{total}")
print(f"  Não processadas: {df_resultado['_dataframeit_status'].isna().sum()}/{total}")

## 9. Analisar Erros em Detalhes

In [ ]:
# Filtrar apenas linhas com erro
linhas_com_erro = df_resultado[df_resultado['_dataframeit_status'] == 'error']

if len(linhas_com_erro) > 0:
    print(f"{len(linhas_com_erro)} linha(s) com erro:\n")

    for idx, row in linhas_com_erro.iterrows():
        print(f"Linha {row['id']}:")
        print(f"  Texto: '{row['texto']}'")
        print(f"  Erro: {row['_error_details']}")
        print()
else:
    print("Nenhum erro encontrado! Todas as linhas foram processadas com sucesso.")

## 10. Estratégias de Recuperação

In [ ]:
# Estratégia 1: Salvar apenas linhas bem-sucedidas
linhas_sucesso = df_resultado[df_resultado['_dataframeit_status'] == 'processed']
print(f"1. Salvar apenas sucesso: {len(linhas_sucesso)} linhas")

# Estratégia 2: Pré-processar dados para evitar erros
print("\n2. Pré-processar dados:")
df_limpo = df.copy()
df_limpo = df_limpo[df_limpo['texto'].str.strip().str.len() > 5]  # Remover textos curtos
print(f"   Após limpeza: {len(df_limpo)} linhas válidas")

## 11. Visualizar Resultados Válidos

In [ ]:
if len(linhas_sucesso) > 0:
    print("Resultados das linhas processadas com sucesso:")
    colunas_exibir = ['id', 'categoria', 'subcategoria', 'preco_estimado']
    display(linhas_sucesso[colunas_exibir])

    print("\nEstatísticas:")
    print("\nDistribuição por categoria:")
    print(linhas_sucesso['categoria'].value_counts())

## 12. Boas Práticas para Tratamento de Erros

**Recomendações:**
- SEMPRE verificar `_dataframeit_status` após o processamento
- Analisar `_error_details` para entender falhas
- Pré-processar dados (remover textos vazios, muito curtos, etc.)
- Usar `resume=True` para poder continuar após interrupções
- Começar com `max_retries` baixo (3) e aumentar se necessário

**Tipos de erro detectados automaticamente:**
- **Recuperáveis**: Timeout, Rate Limit, 5xx - farão retry automaticamente
- **Não-recuperáveis**: API key inválida, 401, 403 - falham imediatamente

---

## Próximos Passos

- [03_resume.ipynb](03_resume.ipynb) - Processamento incremental
- [08_rate_limiting.ipynb](08_rate_limiting.ipynb) - Configurar rate limiting